In [1]:
import os
from glob import glob
import dask
import xarray as xr
import xrft
import cartopy.crs as ccrs
import numpy as np
from netCDF4 import Dataset
from scipy.io import loadmat

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from dask.distributed import Client, LocalCluster
    #
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 251.08 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33739,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 251.08 GiB
Comm: tcp://127.0.0.1:42719,Total threads: 8
Dashboard: http://127.0.0.1:40085/status,Memory: 15.69 GiB
Nanny: tcp://127.0.0.1:38877,


# Grid

In [3]:
# the path where (Zach's) LLC4320 data are already stored
LLC_data_folder = '/meddy/yuxlong5/data/LLC4320/'
# the path where the grid data will be re-stored
LLC_data_folder_data_store = '/meddy/yuxlong5/data/data_zarr/'

In [4]:
XC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/XC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
XC = np.real(XC).rename('XC')

YC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/YC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
YC = np.real(YC).rename('YC')

XG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/XG_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon_g'])
XG = np.real(XG).rename('XG')

YG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/YG_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon_g'])
YG = np.real(YG).rename('YG')

CS = xr.DataArray(np.fromfile(LLC_data_folder+"grid/AngleCS_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
CS = np.real(CS).rename('CS')

SN = xr.DataArray(np.fromfile(LLC_data_folder+"grid/AngleSN_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
SN = np.real(SN).rename('SN')

Depth = xr.DataArray(np.fromfile(LLC_data_folder+"grid/Depth_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
Depth = np.real(Depth).rename('Depth')



dxC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DXC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon_g'])
dxC = np.real(dxC).rename('dxC')

dyC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DYC_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon'])
dyC = np.real(dyC).rename('dyC')

dxG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DXG_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon'])
dxG = np.real(dxG).rename('dxG')

dyG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DYG_177x201", dtype='>f').reshape(201,177), dims=['lat','lon_g'])
dyG = np.real(dyG).rename('dyG')


rA = xr.DataArray(np.fromfile(LLC_data_folder+"grid/RAC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
rA = np.real(rA).rename('rA')

rAz = xr.DataArray(np.fromfile(LLC_data_folder+"grid/RAZ_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon_g'])
rAz = np.real(rAz).rename('rAz')

rAw = xr.DataArray(np.fromfile(LLC_data_folder+"grid/RAW_177x201", dtype='>f').reshape(201,177), dims=['lat','lon_g'])
rAw = np.real(rAw).rename('rAw')

rAs = xr.DataArray(np.fromfile(LLC_data_folder+"grid/RAS_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon'])
rAs = np.real(rAs).rename('rAs')

In [5]:
grid = xr.merge([XC,YC,XG,YG,YC,XG,CS,SN,Depth,dxC,dyC,dxG,dyG,rA,rAz,rAw,rAs])
print(grid)

<xarray.Dataset>
Dimensions:  (lat: 201, lon: 177, lat_g: 201, lon_g: 177)
Dimensions without coordinates: lat, lon, lat_g, lon_g
Data variables: (12/15)
    XC       (lat, lon) >f4 -18.03 -18.01 -17.99 -17.97 ... -14.41 -14.39 -14.36
    YC       (lat, lon) float32 47.4 47.4 47.4 47.4 ... 49.99 49.99 49.99 49.99
    XG       (lat_g, lon_g) float32 -18.04 -18.02 -18.0 ... -14.42 -14.4 -14.38
    YG       (lat_g, lon_g) >f4 47.4 47.4 47.4 47.4 ... 49.98 49.98 49.98 49.98
    CS       (lat, lon) >f4 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    SN       (lat, lon) >f4 -2.519e-13 -0.0 -0.0 ... 2.653e-13 -2.652e-13
    ...       ...
    dxG      (lat_g, lon) >f4 1.568e+03 1.568e+03 ... 1.489e+03 1.489e+03
    dyG      (lat, lon_g) >f4 1.469e+03 1.469e+03 ... 1.403e+03 1.403e+03
    rA       (lat, lon) >f4 2.302e+06 2.302e+06 2.302e+06 ... 2.09e+06 2.09e+06
    rAz      (lat_g, lon_g) >f4 2.303e+06 2.303e+06 ... 2.09e+06 2.09e+06
    rAw      (lat, lon_g) >f4 2.302e+06 2.302e+06 .

In [6]:
lat = YC.isel(lon=1)
lat = np.real(lat).rename('lat')

lon = XC.isel(lat=1)
lon = np.real(lon).rename('lon')

lat_g = YG.isel(lon_g=1)
lat_g = np.real(lat_g).rename('lat_g')

lon_g = XG.isel(lat_g=1)
lon_g = np.real(lon_g).rename('lon_g')

grid = xr.Dataset(grid,coords={'lon':lon, 'lat':lat, 'lat_g':lat_g, 'lon_g':lon_g})
grid

<xarray.Dataset>
Dimensions:  (lat: 201, lon: 177, lat_g: 201, lon_g: 177)
Coordinates:
  * lon      (lon) >f4 -18.03 -18.01 -17.99 -17.97 ... -14.41 -14.39 -14.36
  * lat      (lat) >f4 47.4 47.42 47.43 47.44 47.46 ... 49.95 49.96 49.98 49.99
  * lat_g    (lat_g) >f4 47.4 47.41 47.42 47.44 ... 49.94 49.96 49.97 49.98
  * lon_g    (lon_g) >f4 -18.04 -18.02 -18.0 -17.98 ... -14.42 -14.4 -14.38
Data variables: (12/15)
    XC       (lat, lon) >f4 -18.03 -18.01 -17.99 -17.97 ... -14.41 -14.39 -14.36
    YC       (lat, lon) float32 47.4 47.4 47.4 47.4 ... 49.99 49.99 49.99 49.99
    XG       (lat_g, lon_g) float32 -18.04 -18.02 -18.0 ... -14.42 -14.4 -14.38
    YG       (lat_g, lon_g) >f4 47.4 47.4 47.4 47.4 ... 49.98 49.98 49.98 49.98
    CS       (lat, lon) >f4 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    SN       (lat, lon) >f4 -2.519e-13 -0.0 -0.0 ... 2.653e-13 -2.652e-13
    ...       ...
    dxG      (lat_g, lon) >f4 1.568e+03 1.568e+03 ... 1.489e+03 1.489e+03
    dyG      (lat, lon_g) >f4 1.469e+03 1.469e+03 ... 1.403e+03 1.403e+03
    rA       (lat, lon) >f4 2.302e+06 2.302e+06 2.302e+06 ... 2.09e+06 2.09e+06
    rAz      (lat_g, lon_g) >f4 2.303e+06 2.303e+06 ... 2.09e+06 2.09e+06
    rAw      (lat, lon_g) >f4 2.302e+06 2.302e+06 ... 2.09e+06 2.09e+06
    rAs      (lat_g, lon) >f4 2.303e+06 2.303e+06 ... 2.09e+06 2.09e+06

In [7]:
grid = grid.assign_coords(XC=grid.XC) 
grid = grid.assign_coords(YC=grid.YC) 
grid = grid.assign_coords(XG=grid.XG) 
grid = grid.assign_coords(YG=grid.YG) 
grid = grid.assign_coords(CS=grid.CS) 
grid = grid.assign_coords(SN=grid.SN) 
grid = grid.assign_coords(Depth=grid.Depth) 
grid = grid.assign_coords(dxC=grid.dxC) 
grid = grid.assign_coords(dyC=grid.dyC) 
grid = grid.assign_coords(dxG=grid.dxG) 
grid = grid.assign_coords(dyG=grid.dyG) 
grid = grid.assign_coords(rA=grid.rA) 
grid = grid.assign_coords(rAz=grid.rAz) 
grid = grid.assign_coords(rAw=grid.rAw) 
grid = grid.assign_coords(rAs=grid.rAs) 

grid

<xarray.Dataset>
Dimensions:  (lat: 201, lon: 177, lat_g: 201, lon_g: 177)
Coordinates: (12/19)
    XC       (lat, lon) >f4 -18.03 -18.01 -17.99 -17.97 ... -14.41 -14.39 -14.36
    YC       (lat, lon) float32 47.4 47.4 47.4 47.4 ... 49.99 49.99 49.99 49.99
    XG       (lat_g, lon_g) float32 -18.04 -18.02 -18.0 ... -14.42 -14.4 -14.38
    YG       (lat_g, lon_g) >f4 47.4 47.4 47.4 47.4 ... 49.98 49.98 49.98 49.98
    CS       (lat, lon) >f4 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0 1.0
    SN       (lat, lon) >f4 -2.519e-13 -0.0 -0.0 ... 2.653e-13 -2.652e-13
    ...       ...
    rAw      (lat, lon_g) >f4 2.302e+06 2.302e+06 ... 2.09e+06 2.09e+06
    rAs      (lat_g, lon) >f4 2.303e+06 2.303e+06 ... 2.09e+06 2.09e+06
  * lon      (lon) >f4 -18.03 -18.01 -17.99 -17.97 ... -14.41 -14.39 -14.36
  * lat      (lat) >f4 47.4 47.42 47.43 47.44 47.46 ... 49.95 49.96 49.98 49.99
  * lat_g    (lat_g) >f4 47.4 47.41 47.42 47.44 ... 49.94 49.96 49.97 49.98
  * lon_g    (lon_g) >f4 -18.04 -18.02 -18.0 -17.98 ... -14.42 -14.4 -14.38
Data variables:
    *empty*

In [8]:
file_out = LLC_data_folder_data_store+'Grid.nc'
%time grid.to_netcdf(file_out, mode='w')     

CPU times: user 24.9 ms, sys: 8.04 ms, total: 33 ms
Wall time: 51.5 ms


In [9]:
cluster.scheduler.close()

<coroutine object Scheduler.close at 0x7fb5281b5940>

2025-04-21 11:14:16,946 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-04-21 11:14:16,946 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-04-21 11:14:16,946 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-04-21 11:14:16,946 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-04-21 11:14:16,946 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-04-21 11:14:16,946 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-04-21 11:14:16,947 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-04-21 11:14:16,947 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-04-21 11:14:16,947 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-04